# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

#pd.set_option('display.max_columns', None)

# Datensätze

In [2]:
df_matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
df_players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')

# Hilfsfunktionen

In [3]:
def fillna_0_astype_int64(dataframe:pd.DataFrame, columns:list=["Siege", "Niederlagen"]):

    for column in columns:
        dataframe[column] = dataframe[column].fillna(0).astype("int64")
    
    return dataframe

In [4]:
def siegesquote_funktion(dataframe:pd.DataFrame):

    dataframe["Siegesquote"] = dataframe["Siege"] / (dataframe["Siege"]+ dataframe["Niederlagen"])

    return dataframe

<br>

# Fragestellung 2: 
Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?

In [5]:
df_matches.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

## Sieges- und Niederlagentabelle der deutschen Spieler

In [6]:
#Alle Spiele, bei denene ein deutscher Spieler das Match gewonnen hat
df_players_GER_win = df_matches[df_matches.winner_ioc == "GER"].groupby(["winner_id", "loser_id"]).size().reset_index(name= "wins_count")
df_players_GER_win

,winner_id,loser_id,wins_count
0,100033,100009,1
1,100033,100011,2
2,100033,100019,1
3,100033,100023,1
4,100033,100024,2
...,...,...,...
6531,211046,202510,1
6532,211048,211166,1
6533,211109,100151,1
6534,211109,211119,1


In [7]:
#Alle Spiele, bei denen ein deutscher Spieler das Match verloren hat
df_players_GER_lose = df_matches[df_matches.loser_ioc == "GER"].groupby(["loser_id", "winner_id"]).size().reset_index(name= "loses_count")
df_players_GER_lose


,loser_id,winner_id,loses_count
0,100033,100009,2
1,100033,100022,1
2,100033,100035,2
3,100033,100039,1
4,100033,100050,1
...,...,...,...
6911,211168,202955,1
6912,211271,100135,1
6913,211289,100149,1
6914,211289,211298,1


## Sieges- und Niederlagenbilanz bei Matches, in denen ein deutscher Spieler mitgespielt hat

In [8]:
# Tabellen joinen
df_merge_players_GER = pd.merge(df_players_GER_win, df_players_GER_lose, how='outer',
                                left_on=['winner_id', 'loser_id'], right_on=['loser_id', 'winner_id'])

#winner_id_x und loser_id_y kombinieren
df_merge_players_GER['Spieler_id_deutsch'] = df_merge_players_GER['winner_id_x'].combine_first(df_merge_players_GER['loser_id_y'])

#loser_id_x und winner_id_y kombinieren
df_merge_players_GER['Spieler_id_Gegner'] = df_merge_players_GER['loser_id_x'].combine_first(df_merge_players_GER['winner_id_y'])

# Spalten entfernen
df_merge_players_GER = df_merge_players_GER[['Spieler_id_deutsch', 'Spieler_id_Gegner', 'wins_count', 'loses_count']]

# Spalten umbenennen
df_merge_players_GER.columns = ['Spieler_id_deutsch', 'Spieler_id_Gegner', 'Siege', 'Niederlagen']

#NaN-Werte der Spalte "Niederlagen" und "Siege" durch 0 ersetzen und Datentypen zu "int64" ändern
fillna_0_astype_int64(df_merge_players_GER)
df_merge_players_GER["Spieler_id_deutsch"] = df_merge_players_GER["Spieler_id_deutsch"].astype("int64")
df_merge_players_GER["Spieler_id_Gegner"] = df_merge_players_GER["Spieler_id_Gegner"].astype("int64")

#Spalte Siegesquote hinzufügen
siegesquote_funktion(df_merge_players_GER)


,Spieler_id_deutsch,Spieler_id_Gegner,Siege,Niederlagen,Siegesquote
0,100033,100009,1,2,0.333333
1,100033,100011,2,0,1.000000
2,100033,100019,1,0,1.000000
3,100033,100023,1,0,1.000000
4,100033,100024,2,0,1.000000
...,...,...,...,...,...
11382,211168,202955,0,1,0.000000
11383,211271,100135,0,1,0.000000
11384,211289,100149,0,1,0.000000
11385,211289,211298,0,1,0.000000


<br>

### Namen der Spieler hinzufügen

In [9]:
# Vor- und Nachname des deutschen Spielers hinzufügen
df_merge_players_GER = pd.merge(df_merge_players_GER, df_players[['player_id', 'name_first', 'name_last']],
                                left_on='Spieler_id_deutsch', right_on='player_id', how='left')
df_merge_players_GER = df_merge_players_GER.rename(columns={'name_first': 'Vorname_deutsch', 'name_last': 'Nachname_deutsch'})

# Vor- und Nachname des ausländischen Spielers hinzufügen
df_merge_players_GER = pd.merge(df_merge_players_GER, df_players[['player_id', 'name_first', 'name_last']],
                                left_on='Spieler_id_Gegner', right_on='player_id', how='left')
df_merge_players_GER = df_merge_players_GER.rename(columns={'name_first': 'Vorname_Gegner', 'name_last': 'Nachname_Gegner'})

# Spaltenreihenfolge anpassen
df_merge_players_GER = df_merge_players_GER[["Spieler_id_deutsch", "Vorname_deutsch", "Nachname_deutsch", 
                                             "Spieler_id_Gegner", "Vorname_Gegner", "Nachname_Gegner", 
                                             "Siege", "Niederlagen", "Siegesquote"]]


In [10]:
df_merge_players_GER

,Spieler_id_deutsch,Vorname_deutsch,Nachname_deutsch,Spieler_id_Gegner,Vorname_Gegner,Nachname_Gegner,Siege,Niederlagen,Siegesquote
0,100033,Wilhelm,Bungert,100009,Istvan,Gulyas,1,2,0.333333
1,100033,Wilhelm,Bungert,100011,Torben,Ulrich,2,0,1.000000
2,100033,Wilhelm,Bungert,100019,Wieslaw,Gasiorek,1,0,1.000000
3,100033,Wilhelm,Bungert,100023,Ramanathan,Krishnan,1,0,1.000000
4,100033,Wilhelm,Bungert,100024,Jan Erik,Lundquist,2,0,1.000000
...,...,...,...,...,...,...,...,...,...
11382,211168,Thomas,Sarach,202955,Bill,Tym,0,1,0.000000
11383,211271,Hans,Finkentscher,100135,Onny,Parun,0,1,0.000000
11384,211289,J,Haussels,100149,Toshiro,Sakai,0,1,0.000000
11385,211289,J,Haussels,211298,Rob,Wheatley,0,1,0.000000


<br>

## Gibt es Angstgegner?

In [11]:
#Absteigend nach Niederlagen und aufsteigend nach Siegesquote sortiert
df_filtered = df_merge_players_GER.sort_values(by=["Siegesquote", "Niederlagen"], ascending=[True, False])

#Top 5 Angstgegner
df_filtered[0:5]

,Spieler_id_deutsch,Vorname_deutsch,Nachname_deutsch,Spieler_id_Gegner,Vorname_Gegner,Nachname_Gegner,Siege,Niederlagen,Siegesquote
10277,104259,Philipp,Kohlschreiber,103819,Roger,Federer,0,15,0.0
10221,104252,Florian,Mayer,103819,Roger,Federer,0,8,0.0
8511,101463,Bernd,Karbacher,102446,Andrei,Medvedev,0,7,0.0
10243,104252,Florian,Mayer,104925,Novak,Djokovic,0,7,0.0
10280,104259,Philipp,Kohlschreiber,104214,Igor,Andreev,0,7,0.0


<br>

## Allgemeine Spielstärke jedes Spielers

### df_matches und df_players joinen

In [12]:
#Spielerdaten zum Sieger hinzufügen
df_matches_players = pd.merge(df_matches, df_players, how="left", left_on="winner_id", right_on="player_id")

#Spielerdaten zum Verlierer hinzufügen
df_matches_players = pd.merge(df_matches_players, df_players, how="left", left_on="loser_id", right_on="player_id")

#Spalten umbennen und doppelte rausschmeißen
df_matches_players.rename(columns={"name_first_x":"name_first_winner","name_last_x":"name_last_winner", "dob_x":"dob_winner", "height_x":"height_winner", "wikidata_id_x":"wikidata_winner", 
                                   "name_first_y":"name_first_loser","name_last_y":"name_last_loser", "dob_y":"dob_loser", "height_y":"height_loser", "wikidata_id_y":"wikidata_loser" }, inplace=True)
df_matches_players.drop(["player_id_x", "hand_x", "ioc_x", "player_id_y", "hand_y", "ioc_y"],axis=1, inplace=True)

df_matches_players.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,name_first_winner,name_last_winner,dob_winner,height_winner,wikidata_winner,name_first_loser,name_last_loser,dob_loser,height_loser,wikidata_loser
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,Douglas,Smith,NaN,NaN,NaN,Peter,Ledbetter,19440703.0,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,Louis,Pretorius,19930829.0,NaN,NaN,Maurice,Pollock,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,Cecil,Pedlow,NaN,NaN,NaN,John,Mulvey,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,Tom,Okker,19440222.0,178.0,Q455411,Unknown,Fearmon,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,Armistead,Neely,19470319.0,NaN,Q18085277,Harry,Sheridan,NaN,NaN,NaN


<br>

### Siegesbilanz jedes Spielers

In [13]:
#Alle Siege pro Spieler ID
df_wins = df_matches_players.groupby(["winner_id", "name_first_winner", "name_last_winner"]).size().reset_index(name="Siege")

#Alle Niederlagen pro Spieler ID
df_loses = df_matches_players.groupby(["loser_id", "name_first_loser", "name_last_loser"]).size().reset_index(name="Niederlagen")

#Tabellen joinen, um Siege und Niederlagen pro Spieler ID zu erhalten
df_merge = pd.merge(df_wins, df_loses, how="outer", left_on="winner_id", right_on="loser_id")

#Spalten winner_id und loser_id kombinieren
df_merge['Spieler_id'] = df_merge['winner_id'].combine_first(df_merge['loser_id'])

#Spalten entfernen
df_merge = df_merge[["Spieler_id", "Siege", "Niederlagen"]]

#NaN-Werte der Spalte "Niederlagen" und "Siege" durch 0 ersetzen und Datentyp zu "int64" ändern
fillna_0_astype_int64(df_merge)
df_merge["Spieler_id"] = df_merge["Spieler_id"].astype("int64")

#Spalte Siegesquote hinzufügen
siegesquote_funktion(df_merge)

#Vor- und Nachname hinzufügen
df_merge = pd.merge(df_merge, df_players, how="left", left_on="Spieler_id", right_on="player_id")
df_merge.rename(columns={"name_first":"Vorname", "name_last":"Nachname"}, inplace=True)
df_merge = df_merge[["Spieler_id", "Vorname", "Nachname", "Siege", "Niederlagen", "Siegesquote"]]

df_merge

,Spieler_id,Vorname,Nachname,Siege,Niederlagen,Siegesquote
0,100001,Gardnar,Mulloy,4,6,0.400000
1,100002,Pancho,Segura,9,21,0.300000
2,100003,Frank,Sedgman,26,31,0.456140
3,100004,Giuseppe,Merlo,7,12,0.368421
4,100005,Richard,Gonzalez,154,90,0.631148
...,...,...,...,...,...,...
7183,211800,George,Szakacz,0,1,0.000000
7184,211801,Jochen,Rosner,0,1,0.000000
7185,211803,Heimo,Tschernatsch,0,1,0.000000
7186,211804,Jesus,Oroquieta,0,1,0.000000


<br>

## Liegen die schlechten Ergebnisse gegen die Angstgegner nun an der generellen Spielstärke beider Spieler? 

In [14]:
#Tabellen joinen, um allgemeine Sigesbilanz beider Spieler mit der aus den Spielen, die sie gegeneinander gespielt haben, zu vergleichen
df_merge2 = pd.merge(df_filtered, df_merge, how="left", left_on="Spieler_id_deutsch", right_on="Spieler_id")
df_merge2 = pd.merge(df_merge2, df_merge, how="left", left_on="Spieler_id_Gegner", right_on="Spieler_id")

#Spalten umbenennen
df_merge2.rename(columns={"Siege_x":"Siege_Match", "Niederlagen_x": "Niederlagen_Match", "Siegesquote_x":"Siegesquote_Match",
                          "Siege_y":"Siege_deutscher_Spieler_allg", "Niederlagen_y":"Niederlagen_deutscher_Spieler_allg", "Siegesquote_y":"Siegesquote_deutscher_Spieler_allg",
                          "Siege":"Siege_Gegner_allg", "Niederlagen":"Niederlagen_Gegner_allg", "Siegesquote":"Siegesquote_Gegner_allg"},inplace=True)

#Spalten entfernen
df_merge2 = df_merge2[["Spieler_id_deutsch", "Vorname_deutsch", "Nachname_deutsch", "Siege_deutscher_Spieler_allg" ,"Niederlagen_deutscher_Spieler_allg", "Siegesquote_deutscher_Spieler_allg",
                       "Spieler_id_Gegner","Vorname_Gegner", "Nachname_Gegner", "Siege_Gegner_allg", "Niederlagen_Gegner_allg","Siegesquote_Gegner_allg", "Siege_Match", "Niederlagen_Match", "Siegesquote_Match" ]]

#df_merge2[0:5], um die Top 5 "schlechtesten" Matchkombinationen, in denen ein deutscher Spieler mitspielt, herauszufiltern
df_merge2[0:5]

,Spieler_id_deutsch,Vorname_deutsch,Nachname_deutsch,Siege_deutscher_Spieler_allg,Niederlagen_deutscher_Spieler_allg,Siegesquote_deutscher_Spieler_allg,Spieler_id_Gegner,Vorname_Gegner,Nachname_Gegner,Siege_Gegner_allg,Niederlagen_Gegner_allg,Siegesquote_Gegner_allg,Siege_Match,Niederlagen_Match,Siegesquote_Match
0,104259,Philipp,Kohlschreiber,482,400,0.546485,103819,Roger,Federer,1265,280,0.818770,0,15,0.0
1,104252,Florian,Mayer,244,264,0.480315,103819,Roger,Federer,1265,280,0.818770,0,8,0.0
2,101463,Bernd,Karbacher,136,161,0.457912,102446,Andrei,Medvedev,322,215,0.599628,0,7,0.0
3,104252,Florian,Mayer,244,264,0.480315,104925,Novak,Djokovic,1045,207,0.834665,0,7,0.0
4,104259,Philipp,Kohlschreiber,482,400,0.546485,104214,Igor,Andreev,238,231,0.507463,0,7,0.0


<br>

### Visualisierung der allgemeinen Siegesquoten der Spieler aus den Top 10 "schlechtesten" Matchkombinationen 

In [15]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_merge2[0:10].index,
    y=df_merge2[0:10]["Siegesquote_deutscher_Spieler_allg"],
    name='Deutscher Spieler',
    customdata=df_merge2[0:10][["Vorname_deutsch", "Nachname_deutsch", "Siegesquote_deutscher_Spieler_allg"]],
    hovertemplate=
    '<i>Name</i>: %{customdata[0]} %{customdata[1]}<br>'+
    '<i>Siegesquote</i>: %{customdata[2]:.2f}<br>'+
    '<extra></extra>',
    marker_color='#b2e061'  # Farbe für den deutschen Spieler
))

fig.add_trace(go.Bar(
    x=df_merge2[0:10].index,
    y=df_merge2[0:10]["Siegesquote_Gegner_allg"],
    name='Gegner',
    customdata=df_merge2[0:10][["Vorname_Gegner", "Nachname_Gegner", "Siegesquote_Gegner_allg"]],
    hovertemplate=
    '<i>Name</i>: %{customdata[0]} %{customdata[1]}<br>'+
    '<i>Siegesquote</i>: %{customdata[2]:.2f}<br>'+
    '<extra></extra>',
    marker_color='#fd7f6f'  # Farbe für den Gegner
))

fig.update_layout(barmode='group', title_text='Vergleich allgemeiner Siegesquoten der Spieler')

fig.show()

<br>

## Gibt es analog Lieblingsgegner einzelner deutscher Spieler?

In [16]:
df_filtered2 = df_merge2.sort_values(by=["Siegesquote_Match", "Siege_Match"], ascending=[False, False]).reset_index()

#Top 5 "Lieblingsgegner"
df_filtered2[0:5]

,index,Spieler_id_deutsch,Vorname_deutsch,Nachname_deutsch,Siege_deutscher_Spieler_allg,Niederlagen_deutscher_Spieler_allg,Siegesquote_deutscher_Spieler_allg,Spieler_id_Gegner,Vorname_Gegner,Nachname_Gegner,Siege_Gegner_allg,Niederlagen_Gegner_allg,Siegesquote_Gegner_allg,Siege_Match,Niederlagen_Match,Siegesquote_Match
0,8833,101414,Boris,Becker,716,225,0.760893,101334,Alexander,Volkov,304,256,0.542857,10,0,1.0
1,8857,101414,Boris,Becker,716,225,0.760893,101611,Cedric,Pioline,390,318,0.550847,9,0,1.0
2,7947,100644,Alexander,Zverev,340,151,0.692464,105173,Adrian,Mannarino,245,289,0.458801,7,0,1.0
3,7933,100644,Alexander,Zverev,340,151,0.692464,104731,Kevin,Anderson,357,253,0.585246,6,0,1.0
4,8731,101414,Boris,Becker,716,225,0.760893,100284,Jimmy,Connors,1275,301,0.809010,6,0,1.0


<br>

### Visualisierung der allgemeinen Siegesquoten der Spieler aus den Top 10 "besten" Matchkombinationen 

In [17]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_filtered2[0:10].index,
    y=df_filtered2[0:10]["Siegesquote_deutscher_Spieler_allg"],
    name='Deutscher Spieler',
    customdata=df_filtered2[0:10][["Vorname_deutsch", "Nachname_deutsch", "Siegesquote_deutscher_Spieler_allg"]],
    hovertemplate=
    '<i>Name</i>: %{customdata[0]} %{customdata[1]}<br>'+
    '<i>Siegesquote</i>: %{customdata[2]:.2f}<br>'+
    '<extra></extra>',
    marker_color='#b2e061'  # Farbe für den deutschen Spieler
))

fig.add_trace(go.Bar(
    x=df_filtered2[0:10].index,
    y=df_filtered2[0:10]["Siegesquote_Gegner_allg"],
    name='Gegner',
    customdata=df_filtered2[0:10][["Vorname_Gegner", "Nachname_Gegner", "Siegesquote_Gegner_allg"]],
    hovertemplate=
    '<i>Name</i>: %{customdata[0]} %{customdata[1]}<br>'+
    '<i>Siegesquote</i>: %{customdata[2]:.2f}<br>'+
    '<extra></extra>',
    marker_color='#fd7f6f'  # Farbe für den Gegner
))

fig.update_layout(barmode='group', title_text='Vergleich allgemeiner Siegesquoten der Spieler')

fig.show()

Hier ist ein ähnliches Schema zu erkennen, wie bei den Top 10 Angstgegnern.Bei 9/10 Matchkombinationen in denen der deutsche Spieler eine deutlich größere Anzahl an Siegen als an Niederlagen verzeichnen kann, ist auch die allgemeine Siegesquote des deutschen Spielers höher als die des Gegners. Lediglich bei der Kombination aus Becker gegen Connors hat Becker eine leicht geringere allgemeine Siegesquote, wobei Becker insgesamt 6 Siege und keine Niederlagen gegen Connors verzeichnen kann. Nun führen wir zur Überprüfung erneut eine lineare Regressiond durch...

<br>

### Lineare Regression zur Überprüfung was den größten Einfluss auf die Anzahl der Siege eines deutschen Spielers (gegen einen bestimmten Konkurrenten) hat

In [18]:
#Features
X = df_merge2[['Siege_deutscher_Spieler_allg', 'Siege_Gegner_allg', "Niederlagen_deutscher_Spieler_allg", "Niederlagen_Gegner_allg", "Siegesquote_deutscher_Spieler_allg", "Siegesquote_Gegner_allg" ]]
#Label
y = df_merge2['Siegesquote_Match']
model = LinearRegression()
model.fit(X, y)

coefficients = model.coef_

for feature, coefficient in zip(X.columns, coefficients):
    print(f"{feature}: {coefficient}")

Siege_deutscher_Spieler_allg: 1.0356971678587171e-05
Siege_Gegner_allg: 6.038456307894927e-05
Niederlagen_deutscher_Spieler_allg: 0.00019827154639354984
Niederlagen_Gegner_allg: -0.00031543578396690656
Siegesquote_deutscher_Spieler_allg: 1.0075383938949136
Siegesquote_Gegner_allg: -1.0716980626195445


Auf Grundlage der Koeffizienten kann man sagen, dass die allgemeine Siegesquote des deutschen Spielers den größten positiven und die des Gegners den größten negativen Einfluss auf die Siegesquote für den deutschen Spieler in der jeweiligen Matchkombination hat.

Daraus kann man vorerst schließen, dass z.B. die schlechte Bilanz von 0-7 von Kohlschreiber gegen Anreev, trotz der höheren allgemeinen Siegesquote von Kohlschreiber, ein Ausreißer zu sein scheint.

Auch die gute Bilanz von 6-0 von Becker gegen Connors scheint trotz der niedrigeren allgemeinen Siegesquote von Becker ein Ausreißer zu sein.

Es könnte sein, dass Kohlschreiber und Becker häufiger sehr starke Gegner bekommen hat und daher seine Siegesquote geringer als die von Andreev ist.

<br>